# Model Design

My model will be a RoBERTa-based model from the Huggingface Transformer library. The architecture is as follows:
1. Fine-tune a RoBERTa model on the claim and explanation fields
2. Use an AdamW optimizer to update the weights
3. Run for 10 epochs with a batch size of 8

In [1]:
# Installations
!pip3 install datasets --quiet
!pip3 install transformers --quiet
from IPython import display # for limiting some display sizes

### Dataset load and inspection

In [2]:
from datasets import list_datasets, load_dataset, Value, ClassLabel

LABEL_DICTIONARY = {0: 'false', 1: 'mixture', 2: 'true', 3: 'unproven'}

dataset = load_dataset('health_fact', 'binary')
print(dataset.keys())
for k in dataset.keys():
    new_id = list(range(len(dataset[k])))
    dataset[k].add_column('new_id', new_id)
print(dataset['train'].features)
print(dataset['test'].features)
print('\n')
dataset['test'][0]

Using custom data configuration binary
Reusing dataset health_fact (/root/.cache/huggingface/datasets/health_fact/binary/1.1.0/99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19)


  0%|          | 0/3 [00:00<?, ?it/s]

dict_keys(['train', 'test', 'validation'])
{'claim_id': Value(dtype='string', id=None), 'claim': Value(dtype='string', id=None), 'date_published': Value(dtype='string', id=None), 'explanation': Value(dtype='string', id=None), 'fact_checkers': Value(dtype='string', id=None), 'main_text': Value(dtype='string', id=None), 'sources': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=4, names=['false', 'mixture', 'true', 'unproven'], id=None), 'subjects': Value(dtype='string', id=None)}
{'claim_id': Value(dtype='string', id=None), 'claim': Value(dtype='string', id=None), 'date_published': Value(dtype='string', id=None), 'explanation': Value(dtype='string', id=None), 'fact_checkers': Value(dtype='string', id=None), 'main_text': Value(dtype='string', id=None), 'sources': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=4, names=['false', 'mixture', 'true', 'unproven'], id=None), 'subjects': Value(dtype='string', id=None)}




{'claim': 'A mother revealed to her child in a letter after her death that she had just one eye because she had donated the other to him.',
 'claim_id': '33456',
 'date_published': 'November 6, 2011',
 'explanation': 'The one-eyed mother story expounds upon two moral messages: the unconditional, all-encompassing love we expect mothers to always feel for their children, and the admonition to not put off cherishing loved ones and appreciating their sacrifices while they’re still around.',
 'fact_checkers': 'David Mikkelson',
 'label': 0,
 'main_text': "In April 2005, we spotted a tearjerker on the Internet about a mother who gave up one of her eyes to a son who had lost one of his at an early age. By February 2007 the item was circulating in e-mail in the following shortened version:  My mom only had one eye. I hated her… She was such an embarrassment. She cooked for students and teachers to support the family. There was this one day during elementary school where my mom came to say hell

Upon further inspection we can find some peculiarities in the labels of this dataset:

In [3]:
label_dict = {-1:0, 0: 0, 1: 0, 2: 0, 3: 0}
count = 0
for i in list(dataset['train']['label']):
    label_dict[i] += 1
    if i == -1: print(count, end =" ")
    count += 1
print()

val_label_dict = {-1:0, 0: 0, 1: 0, 2: 0, 3: 0}
for i in list(dataset['validation']['label']):
    val_label_dict[i] += 1

test_label_dict = {-1:0, 0: 0, 1: 0, 2: 0, 3: 0}
for i in list(dataset['test']['label']):
    test_label_dict[i] += 1

print(label_dict)
print(val_label_dict)
print(test_label_dict)

1312 1313 1314 2151 2152 2478 2479 2480 2650 2651 2652 5259 5260 5261 5489 5490 5491 6053 6054 6055 6216 6217 6218 6654 6655 8506 8507 8508 
{-1: 28, 0: 3001, 1: 1434, 2: 5078, 3: 291}
{-1: 11, 0: 380, 1: 164, 2: 629, 3: 41}
{-1: 2, 0: 388, 1: 201, 2: 599, 3: 45}


In the below, the -1 labels have one thing that seem to be consistent among them: they do not correspond to an actual medical claim. It seems to be more about logistics, or the information is missing altogether. As they represent a small part of the dataset we remove these labels.

In [4]:
print(dataset['train']['claim'][1312:1315])
print(dataset['train']['explanation'][1312:1315])

# Filter the dataset to keep only the labels that make sense
dataset = dataset.filter(lambda example: example['label'] > -0.5)

Loading cached processed dataset at /root/.cache/huggingface/datasets/health_fact/binary/1.1.0/99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19/cache-4f9558a30922a2c4.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/health_fact/binary/1.1.0/99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19/cache-29b1405e3085ad2c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/health_fact/binary/1.1.0/99503637e4255bd805f84d57031c18fe4dd88298f00299d56c94fc59ed68ec19/cache-7f4c8737cf0a5f22.arrow


[' A forwarded email that cautions veterans about   questions that maybe asked while in treatment at the Veterans Hospital.\xa0 The email says that   if a veteran is asked about being stressed, feeling threatened or   wanting to harm someone and if their answer is yes they could lose any   authorization to carry a concealed weapon.\xa0 A warning to those to   expect healthcare workers to ask, “Do you have a gun in your house.” \xa0       ', '', 'This one is fiction. A   spokesperson at the Veterans Hospital told TruthorFiction.com that all patients are protected by doctor/patient   confidentiality and any information taken by the Veterans Administration   (VA) is never shared with any outside agencies. The only   exemption to the case would be if a hospital were asked to assist in a   criminal investigation. The three questions about feeling   stressed, threatened and desiring to harm others are to test for Post   Traumatic Stress Disorder (PTSD) which some of our men and women in the 

### Load Roberta Tokenizer and Model, and prepare dataset for PyTorch

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import torch

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=4)

def tokenize_function(batch):
    encoded = tokenizer(batch["claim"], batch["explanation"], padding="max_length", truncation=True, max_length=512)
    return encoded

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["claim_id", "date_published", "fact_checkers", "main_text", "sources", "subjects"])
tokenized_dataset = tokenized_dataset.remove_columns(["claim", "explanation"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")
print(tokenized_dataset['train'].features)

# Use DataLoader for batched input to PyTorch
batch_size = 8
train_dataloader = DataLoader(tokenized_dataset['train'], shuffle=True, batch_size=batch_size)
val_dataloader = DataLoader(tokenized_dataset['validation'], shuffle=True, batch_size=batch_size)
test_dataloader = DataLoader(tokenized_dataset['test'], shuffle=True, batch_size=batch_size)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

{'labels': ClassLabel(num_classes=4, names=['false', 'mixture', 'true', 'unproven'], id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


### Setting up optimizers and training the model

In [6]:
from transformers import AdamW
from transformers import get_scheduler

# Optimizer and schedules for optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 10
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Setup GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(model.to(device))
display.Javascript("google.colab.output.setIframeHeight('300px');") # limit the display size

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

<IPython.core.display.Javascript object>

Now we train the model: (We first train the model with no class reweighting. This should give us good training accuracies but not as great generalization to the validation and test set.)

In [7]:
from tqdm.auto import tqdm
from datasets import load_metric
from torch.nn import CrossEntropyLoss
import random

progress_bar = tqdm(range(num_training_steps))

from transformers import AutoModelForSequenceClassification, Trainer

row_format ="{:<10} {:<20} {:<15} {:<20} {:<15}"
print(row_format.format("", "Approx. train loss", "Val loss", "Approx. train acc", "Val acc"))

# Train Model
model.train()
for epoch in range(num_epochs):
    num_batches = 0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        num_batches+=1

        # Every 200 batches (1600 samples) print metrics
        if num_batches%200 == 0:
            model.eval()

            with torch.no_grad():
                metric1_train = load_metric("accuracy")
                metric1_val = load_metric("accuracy")
                #metric2 = load_metric("f1")

                # compute train loss
                train_loss = 0
                rand_loss_iters = set(random.sample(range(len(train_dataloader)), len(train_dataloader)//10)) # eval 0.1 of training set for approx.
                count = -1
                for batch in train_dataloader:
                    count += 1
                    if count not in rand_loss_iters: continue
                    batch = {k: v.to(device) for k, v in batch.items()}

                    outputs = model(**batch)
                    train_loss += outputs.loss/len(rand_loss_iters)

                    predictions = torch.argmax(outputs.logits, dim=-1)
                    metric1_train.add_batch(predictions=predictions, references=batch["labels"])

                # compute val loss and metrics
                val_loss = 0
                for batch in val_dataloader:
                    batch = {k: v.to(device) for k, v in batch.items()}

                    outputs = model(**batch)
                    val_loss += outputs.loss/len(val_dataloader)

                    logits = outputs.logits
                    predictions = torch.argmax(logits, dim=-1)
                    metric1_val.add_batch(predictions=predictions, references=batch["labels"])

                print(row_format.format(f"Step {num_batches}", f"{train_loss:.6f}", f"{val_loss:.6f}", f"{metric1_train.compute()['accuracy']:.6f}", f"{metric1_val.compute()['accuracy']:.6f}"))
            model.train()

#torch.save(model.state_dict(), PATH_if_colab_or_not)

  0%|          | 0/12260 [00:00<?, ?it/s]

           Approx. train loss   Val loss        Approx. train acc    Val acc        
Step 200   0.795227             0.798312        0.649590             0.670511       
Step 400   0.810914             0.787706        0.646516             0.664745       
Step 600   0.849897             0.833256        0.588115             0.641680       
Step 800   0.777563             0.814706        0.655738             0.664745       
Step 1000  0.844570             0.828513        0.593238             0.642504       
Step 1200  0.910838             0.974191        0.549180             0.518122       
Step 200   0.884121             0.848163        0.627049             0.654036       
Step 400   0.823827             0.838586        0.637860             0.636738       
Step 600   0.758349             0.795792        0.688525             0.670511       
Step 800   0.762162             0.746002        0.645492             0.675453       
Step 1000  0.706068             0.740307        0.725309         

As we can see our model starts overfitting the training set, achieving near 98% accuracy, but due to our decreasing learning rates sees a nice plateauing of the validation accuracies. We get a humbly moderate validation accuracy of around 74%. Due to time constraints with training the transformer, I could not test more settings, but my first bet would be to perhaps set the learning rate scheduler to be exponential step function. Then, I would try decreasing the complexity of the model by encouraging more drop out in the final layers of RoBERTa. A final strategy could be to encode both sentences separately so that we can get a max sequence length of 1024, and inputting both encodings into another neural network.

Here we calculate the test accuracy and f1 scores:

In [13]:
metric_acc = load_metric("accuracy")
metric_false_f1 = load_metric("f1")
metric_mixture_f1 = load_metric("f1")
metric_true_f1 = load_metric("f1")
metric_unproven_f1 = load_metric("f1")
model.eval()

progress_bar = tqdm(range(len(test_dataloader)))

# compute metrics for the test set
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric_acc.add_batch(predictions=predictions, references=batch["labels"])
        metric_false_f1.add_batch(predictions=(predictions == 0), references=(batch["labels"] == 0))
        metric_mixture_f1.add_batch(predictions=(predictions == 1), references=(batch["labels"] == 1))
        metric_true_f1.add_batch(predictions=(predictions == 2), references=(batch["labels"] == 2))
        metric_unproven_f1.add_batch(predictions=(predictions == 3), references=(batch["labels"] == 3))
    progress_bar.update(1)

for metric in ["metric_acc", "metric_false_f1", "metric_mixture_f1", "metric_true_f1", "metric_unproven_f1"]:
    metric_dict = globals()[metric].compute()
    print(f"{metric[7:]}: {metric_dict[list(metric_dict.keys())[0]]:.6f}")

  0%|          | 0/155 [00:00<?, ?it/s]

acc: 0.704785
false_f1: 0.675159
mixture_f1: 0.444444
true_f1: 0.843561
unproven_f1: 0.433735


We get an acceptable accuracy of around 70%. Looking at the f1 scores, we see that we get the highest f1 for true claims, and the second highest accuracy for false claims. This is good as when we receive a useful or harmful claim, our model has a good chance of knowing the correctness of those claims. Our model is more ambiguous on mixed or unproven claims, though this makes sense due to adversarial nature of mixed claims and the class imbalance with unproven claims. Methods to counter this could be to do contradiction detection on the explanation, such as in this [paper](https://nlp.stanford.edu/pubs/contradiction-acl08.pdf), and to introduce some reweighting of examples as I have set the bare bones of below.

In [ ]:
# inverse_weight_tensor = torch.zeros(3)
# for i in range(4): inverse_weight_tensor[i] = 1/label_dict[i]
# inverse_weight_tensor = inverse_weight_tensor/torch.sum(inverse_weight_tensor)*3

# sqrt_weight_tensor = torch.zeros(3)
# for i in range(4): sqrt_weight_tensor[i] = 1/(label_dict[i]**0.5)
# sqrt_weight_tensor = sqrt_weight_tensor/torch.sum(sqrt_weight_tensor)*3

# When calculating loss:
    # loss_reweight = torch.Tensor([inverse_weight_tensor[label] for label in batch["labels"]]).to(device)
    # batch = {k: v.to(device) for k, v in batch.items()}
    # outputs = model(**batch)
    # logits = outputs.logits
    # loss_fct = CrossEntropyLoss(weight=loss_reweight)
    # loss = loss_fct(logits, batch["labels"])
    # loss.backward()